In [3]:
!pip install duckdb
import pandas as pd
import duckdb

# 1. 读取数据（Online Retail 数据常用 latin1 编码）
df = pd.read_csv('onlineretail.csv', encoding='latin1')

# 2. 基本清洗（避免计算错误）
df = df.dropna(subset=['CustomerID'])         # 去掉 CustomerID 为空的行
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]                   # 去掉退货或无效数量（可选） # 去掉免费或异常价格
df['CustomerID'] = df['CustomerID'].astype(int)  # 转为整数（便于处理）


# 拆分虚拟表
customers = df[['CustomerID', 'Country']].drop_duplicates()  # 去重：每个客户只保留一行   内层 [...] 是选择列列表，外层 [...] 是 DataFrame 的列选择
orders = df[['InvoiceNo', 'CustomerID', 'StockCode', 'Quantity', 'UnitPrice', 'InvoiceDate']]

query1 = """

SELECT 
    o.InvoiceNo,          -- 从订单表取字段（别名 o）
    c.CustomerID,         -- 从客户表取字段（别名 c）
    c.Country,
    o.Quantity,
    o.UnitPrice
FROM orders o
INNER JOIN customers c
 ON o.CustomerID = c.CustomerID
LIMIT 20;


"""
# 4. 执行 SQL 并查看结果
df_inner=duckdb.query(query1).df()
display.df_inner()
query2 = """

SELECT 
    c.CustomerID,
    c.Country,
    o.InvoiceNo           -- 如果某客户没订单，这里会是 NULL
FROM customers c          -- 左表：customers
LEFT JOIN orders o        -- LEFT JOIN：保留左表所有行
  ON c.CustomerID = o.CustomerID
LIMIT 100;


"""

df_left=duckdb.sql(query2).df()
print(df_left.drop_duplicates() )

AttributeError: 'function' object has no attribute 'df_inner'